In [1]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool
import os

In [2]:
os.environ["OPENAI_API_KEY"] = "NA"

# search_tool = SerperDevTool()

In [3]:
design_model = Ollama(model = "llama3:70b-instruct")
woker_model = Ollama(model = "codestral:22b")

In [4]:
game_designer = Agent(
  role='游戏设计师',
  goal='详细描述游戏{topic}的规则，提供给开发者用于开发',
  verbose=True,
  memory=True,
  backstory=(
    "你是资深的游戏设计师。"
    "你热爱并熟悉所有经典游戏，对游戏规则熟悉了然于胸。"
  ),
  # tools=[search_tool],
  allow_delegation=False,
  llm=design_model
)

worker = Agent(
  role='游戏开发者',
  goal='用html开发游戏{topic}',
  verbose=True,
  memory=True,
  backstory=(
    "你是一名资深的游戏开发者。"
    "你擅长用游戏设计师设计的游戏规则来构建游戏。"
    "你精通用html、javascript、css开发游戏。"
  ),
  # tools=[search_tool],
  allow_delegation=False,
  llm=woker_model
)

In [5]:
game_design_task = Task(
  description=(
    "描述游戏{topic}的开发需求。"
    "需求中需要包含游戏界面的详细描述，界面精美，能满足游戏玩家。"
    "需求中需要详细描述游戏规则，包括并不限于游戏玩家数量、游戏启动、得分、游戏结束、游戏失败的规则。"
  ),
  expected_output='用Markdown格式输出游戏的需求和规则',
  agent=game_designer,
  # tools=[search_tool],
)

worker_task = Task(
  description=(
    "用html、javascript、css、图片和必要的三方件构建游戏{topic}。"
    "游戏需要满足设计师提供的需求和规则。"
    "请输出游戏的详细代码。"
  ),
  expected_output='仅输出游戏的详细代码，html和javascript、css代码都放在同一个html文件中，不需要输出代码注释和其他解释说明内容',
  agent=worker,
  # tools=[search_tool],
  async_execution=False,
  output_file='output/game_snake.html'
)

In [6]:
crew = Crew(
  agents=[game_designer, worker],
  tasks=[game_design_task, worker_task],
  process=Process.sequential,
#   memory=True,
#   cache=True,
  max_rpm=5,
  share_crew=True
)

In [7]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': '贪吃蛇'})
print(result)



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer 

Final Answer:

**Game Title:** Greedy Snake
**Game Overview:** A classic arcade-style game where players control a snake that grows in length as it consumes food pellets. The goal is to accumulate points while avoiding collisions with walls, itself, and other snakes (in multiplayer mode).

**Game Interface:**
### Game Board:
The game board consists of a rectangular grid with a fixed size (e.g., 20x20). The grid is divided into squares, each representing a single unit of movement.

### UI Elements:

* **Snake:** A series of connected squares that represent the snake's body. Each square has a distinct color and design to differentiate it from other game elements.
* **Food Pellets:** Small, circular icons scattered throughout the grid, serving as the primary source of points.
* **Walls:** Thick, dark lines that form the boundaries of the grid, preventing the snake from escaping or moving outside the playa